# Collocation Networks

For my presentation at SDSS I depended upon Wordij to generate a collocation network for the text I was treating — I was demonstrating the focus on a single text that has emerged recently in corpus stylistics — along with its companion application, Visij. In the days leading up to the conference, I looked around for a Python equivalent and was, honestly, surprised not to be able to find anything. In these days following the conference, I have looked some more and I have still been unable to find anything. The only software I have seen is a part of the Lancashire group's efforts: they also produced LancsBox and I signed up for the course. (There's something I can do in the weeks ahead!)

I found myself thinking about writing my own script, or scripts to do it. As I have thought about it, I have imagined the mechanics would looks little like what is described below.

Just now, as I was writing this, I realized I have looked mostly for "Python collocation network" but I have not looked for scripts that are simply "Python collocation". I also realized that I could potentially either use, or base my own work, on the functionality built into the NLTK's `ngram` modules.

### Danowski on "Conceptualizing Semantic Networks"

The following is taken from:

Danowki, J. A. 2011. Counter-Terrorism Mining for Individuals Semantically Similar to Watchlist Members. In _Counter-Terrorism and Open Source Intelligence_, 223-248. Ed. Uffe Kock Wiil. Springer. 

> Imagine a large group of people using the same language over time. Assume that the full text of their messages is available to you in natural language form. How would you come to some representation of what they are talking about? Your first thought may be to use traditional content analysis methods that categorize text, either manual procedures or computerized ones
like the General Inquirer [1] or its more recent cousin LIWC [ ], or topic modeling software based on Bayesian statistical procedures [ ]. These automated procedures, while computationally sophisticated, are relatively crude at the conceptual level. They merely assign message elements or individuals to a limited number of nominal categories.
> 
> Instead of categorizing messages, with a network perspective one can capture the relationships among words within the messages. Defining word-pair link strength as the number of times each word occurs closely in text with another, all possible word pairs have an occurrence distribution whose values range from zero on up. This ratio scale of measurement allows the use of sophisticated statistical tools from social network analysis toolkits. These enable the mapping of the structure of the word network. They identify word groups, or clusters, And quantify the structure of the network at different levels. Using these word-pair data as input to network analysis tools, you map the language landscape. On the map, instead of cities, the nodes are words. Rather than roads, there are links or edges among words.
> 
> Travelling through the word network are fleets of social objects. These communication vehicles are the concepts, ideas, or physical things that people linguistically describe. As they link words to these vehicles in the course of their everyday informal and formal communication, this propels them
through the network. Sometimes these movements are unplanned. At other times, groups or organizations try to manage vehicular traffic. By means of optimal messages, they try to steer vehicles in the flow of traffic away from certain words or toward them [2].
> 
> Mathematically-based procedures have been developed to create optimal messages. These are constructed through systematic analysis of the paths connecting word nodes interest. The procedures identify the optimal association network across the aggregate social community. The underlying assumption is that stimulating associations across it is more effective as the shortest effective sequence of words based on particular constraints is selected for the message. This is because people process strings of words linearly over time, encoding and decoding them in sequences. Furthermore, the triggering of associations to words in context takes cognitive time. The most effective message, therefore, optimizes the association networks in the receivers' minds as they read or hear the message.
> 
> In short, this paper focuses on the similarity of messages encoded by individuals, in other words, individuals’ semantic network similarity. This constitutes a new kind of network variable, in addition to cohesion based on actual communication exchange, and in addition to structural equivalence based on similarity of network position. This new network variable is semantic equivalence. Some may think this construct means entities have the same linguistic code identifier, such as the same name for a person, organization, or object. These are not semantic network characteristics but semantic attributes of some entity. They are akin to the words in a dictionary or elements of an ontology. In contrast, one can consider the networks among semantic elements encoded by persons or other social units. Our interest is in semantic encoding similarity.

## Early Build Notes

  The "window" is the same thing as an ngram, but reducing the gram to simply the relationship between the base word, here the word furthest to the right, and its collocates. **Question**: it is at all interesting how close or far the words are? That is, do we want to know if a word is three steps or four steps away? **Tentative answer**: no.

So, we are keeping track of word pairs, counting the number of times two words are within some distance of each other. We have:

    base, collocate, frequency

We can store that as a tuple or as a Pandas dataframe (which is essentially a Numpy array?).

From [http://compling.hss.ntu.edu.sg/courses/hg2051/week09.html], there is the following:

```python
def collocations(words):
    from operator import itemgetter

    # Count the words and bigrams

    wfd = nltk.FreqDist(words)
    pfd = nltk.FreqDist(tuple(words[i:i+2]) for i in range(len(words)-1))

    #
    scored = [((w1,w2), score(w1, w2, wfd, pfd)) for w1, w2 in pfd]
    ## sort according to the score
    scored.sort(key=itemgetter(1), reverse=True)
    return [p for (p,s) in scored]


def score(word1, word2, wfd, pfd, power=3):
    'return the collocation score f(w1,w2)^power/(f(w1)*f(w2))'''
    freq1 = wfd[word1]
    freq2 = wfd[word2]
    freq12 = pfd[(word1, word2)]
    return freq12 ** power / float(freq1 * freq2)
```


Part of a [course](http://compling.hss.ntu.edu.sg/courses/hg2051/) being offered by [Francis Bond](http://www3.ntu.edu.sg/home/fcbond/) at Nanyang Technological University. One the page for the course, he cites Jurafsky and Martin's _[Speech and Language Processing](http://www.cs.colorado.edu/~martin/slp.html)_ (2009).

---

**NLTK Solution?** According to the NLTK documentation on [Collocations](http://www.nltk.org/howto/collocations.html), one can set a window for bigrams. *Duh!* 

```python
finder = BigramCollocationFinder.from_words(
    nltk.corpus.genesis.words('english-web.txt'),
    window_size = 20)
finder.apply_freq_filter(2)
ignored_words = nltk.corpus.stopwords.words('english')
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
# doctest: +NORMALIZE_WHITESPACE
finder.nbest(bigram_measures.likelihood_ratio, 10)
```

Contiguous word collocations as directed networks get fairly close to discourse in a very compelling way. For too long, I have depended on a Java application, Wordij, both to get the data and to visualize it. What follows is my attempt to build a working codebase in Python.

What the program needs to do:

* accept a text or group of texts as input
* break those strings into a series of word pairs (ideally this would include a measure of how far apart the words were)
* compile the word pairs into a series of nodes and edges such that `word1 > word2` with a count of how many times that pair arises so that the edges can be weighted
* visualize the network

Parameters:

* window size
* include/exclude stopwords
* network dimensions (thresholds to be included, various network optimizations)

Useful links:

* [NLTK documentation on collocations](http://www.nltk.org/howto/collocations.html)
* [Getting the count of a collocation](https://stackoverflow.com/questions/38597503/in-nltk-get-the-number-of-occurrences-of-a-trigram)
* This [SO thread][so] lays out the reasons for assembling all bigrams first and then filtering the less interesting ones out later. 

[so]: https://stackoverflow.com/questions/19145332/nltk-counting-frequency-of-bigram

In [5]:
import re

In [1]:
import nltk
from nltk.collocations import *

In [7]:
# First, we need a text:
mdg = open('texts/mdg.txt', 'r').read()
words = re.sub("[^a-zA-Z']"," ", mdg).lower().split()

We begin with code straight from the NLTK:

In [30]:
window = 15
pairs = 5
ignore_pairs = 2
ignored_words = nltk.corpus.stopwords.words('english')

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.BigramCollocationFinder.from_words(words, window_size = window)
finder.apply_freq_filter(ignore_pairs)
finder.apply_word_filter(lambda w: w in ignored_words) # len(w) < 3 or w.lower()
finder.nbest(bigram_measures.likelihood_ratio, pairs)

[('nearer', 'nearer'),
 ('general', 'zaroff'),
 ('drew', 'nearer'),
 ('nearer', 'ridge'),
 ('mr', 'rainsford')]

I don't entirely understand the "measures" being applied above. I know it's in the MLTK documentation, and in the NLP literature in general, but I would like to start my work here by getting a count.

In [26]:
my_finder = nltk.BigramCollocationFinder.from_words(words, window_size = 3)

In [ ]:
for k,v in my_finder.ngram_fd.items():
    print(k,v)

The line above outputs the follow:

    ('off', 'there') 1
    ('off', 'to') 2
    ('there', 'to') 2
    ('there', 'the') 3
    ('to', 'the') 25
    ('to', 'right') 2
    ('the', 'right') 4

I tried converting to a list with:

    bgm_list = [ "({}), {}"%(k,v) for k,v in my_finder.ngram_fd.items()]

In [44]:
bigrams = my_finder.ngram_fd

In [29]:
type(bigrams)

nltk.probability.FreqDist

### FreqDist as Counter

What we have is an NLTK Frequency Distribution. The FreqDist object in NLTK is a sub-class of the native Python's `collections.Counter`, which are dictionaries that store the elements in a list as its key and the counts of the elements as the values ([URL][]). They work like this:

[URL]: https://stackoverflow.com/questions/37427673/sorting-freqdist-in-nltk-with-get-vs-get

In [36]:
from collections import Counter

Counter(['a','a','b','c','c','c','d'])

Counter({'a': 2, 'b': 1, 'c': 3, 'd': 1})

In [37]:
c = Counter(['a','a','b','c','c','c','d'])
c.most_common()

[('c', 3), ('a', 2), ('b', 1), ('d', 1)]

In [39]:
list(reversed(c.most_common()))

[('d', 1), ('b', 1), ('a', 2), ('c', 3)]

In [38]:
[key for key in c]

['a', 'b', 'c', 'd']

In [40]:
c.keys()

dict_keys(['a', 'b', 'c', 'd'])

In [41]:
c.items()

dict_items([('a', 2), ('b', 1), ('c', 3), ('d', 1)])

### Getting Data out of the `FreqDist`

In [ ]:
bigrams.most_common()

Outputs the following:

```
[(('the', 'general'), 74),
 (('of', 'the'), 51),
 (('the', 'of'), 50),
 (('in', 'the'), 37),
 (('the', 'and'), 28),
 (('it', 'was'), 27),
 (('to', 'the'), 25),
 (('a', 'of'), 25),
 (('he', 'was'), 24),
 (('the', 'was'), 22),
 (('he', 'the'), 22),
 (('was', 'the'), 22),
 (('rainsford', 'the'), 21),
 ...]
 
 ```

So, we have a list of tuples, each consisting of a tuple and an integer.

Now we can return to the code above:

```python
for k,v in finder.ngram_fd.items():
    print(k,v)
```

with the understanding that the pair of words, the bigram, is the key and the count is the value. 

The code below is adapted from a question about [NLTK trigrams](https://stackoverflow.com/questions/38597503/in-nltk-get-the-number-of-occurrences-of-a-trigram). 

In [51]:
bgm_counts = sorted(my_finder.ngram_fd.items(), key=lambda t: (-t[1], t[0]))

In [52]:
len(bgm_counts)

12234

In [53]:
print(bgm_counts[0:5])

[(('the', 'general'), 74), (('of', 'the'), 51), (('the', 'of'), 50), (('in', 'the'), 37), (('the', 'and'), 28)]


## Create the Network

See: 
* [Exploring and Analyzing Network Data with Python | Programming Historian](https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python)
* And it looks like loading a graph in NetworkX is easier than I imagined: [Tutorial — NetworkX 2.4 documentation](https://networkx.github.io/documentation/stable/tutorial.html#adding-attributes-to-graphs-nodes-and-edges).